In [1]:
import preprocessing.sinonom_pdf_helper as sn
import preprocessing.vietnamese_pdf_helper as vn
import preprocessing.translate_sn2vn
import alignment
import ocr_correction.corrector as crt
import os
import random
import argparse
from random import seed as seed
from laserembeddings import Laser
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re

/home/davidle2810/anaconda3/envs/mm_align/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/davidle2810/anaconda3/envs/mm_align/lib/python3.10/site-packages/laserembeddings/encoder.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user v

In [2]:
laser = Laser()

In [3]:
sn_content = [{'page_number': 0, 'content': [{'bbox': [[3063, 933], [3356, 929], [3363, 2624], [3071, 2628]], 'content': '型敎要理目錄', 'transliteration': 'hình giáo yếu lý mục lục '}, {'bbox': [[2733, 1187], [2994, 1183], [3002, 2340], [2740, 2344]], 'content': '天主一體', 'transliteration': 'thiên chúa nhất thể '}, {'bbox': [[2840, 3293], [3044, 3289], [3052, 4161], [2844, 4165]], 'content': '見一張', 'transliteration': 'kiến nhất trương '}, {'bbox': [[2398, 1206], [2671, 1206], [2671, 2332], [2398, 2332]], 'content': '降生救世', 'transliteration': 'giáng sanh cứu thế '}, {'bbox': [[2514, 3289], [2698, 3285], [2706, 4146], [2521, 4150]], 'content': '見三張', 'transliteration': 'kiến tam trương '}, {'bbox': [[2179, 430], [2237, 430], [2237, 553], [2179, 553]], 'content': '4', 'transliteration': '4 '}, {'bbox': [[2056, 1206], [2318, 1202], [2333, 2324], [2068, 2328]], 'content': '死期有定', 'transliteration': 'tử kỳ hữu định '}, {'bbox': [[2156, 3293], [2356, 3289], [2368, 4176], [2168, 4180]], 'content': '見六張', 'transliteration': 'kiến lục trương '}, {'bbox': [[1722, 1198], [1999, 1195], [2006, 2324], [1733, 2328]], 'content': '人生罪種', 'transliteration': 'nhân sinh tội giống '}, {'bbox': [[1860, 3297], [2045, 3297], [2045, 4176], [1860, 4176]], 'content': '見八張', 'transliteration': 'kiến bát trương '}, {'bbox': [[1364, 1198], [1653, 1191], [1687, 2332], [1395, 2344]], 'content': '聖秘跡論', 'transliteration': 'thánh bí tích luận '}, {'bbox': [[1499, 3297], [1683, 3293], [1703, 4169], [1514, 4173]], 'content': '十一張', 'transliteration': 'thập nhất trương '}, {'bbox': [[1030, 1187], [1322, 1187], [1322, 2344], [1030, 2344]], 'content': '聖體解論', 'transliteration': 'thánh thể giải luận '}, {'bbox': [[1160, 3300], [1349, 3300], [1349, 4173], [1160, 4173]], 'content': '十四張', 'transliteration': 'thập tứ trương '}, {'bbox': [[672, 1183], [957, 1179], [964, 2336], [680, 2340]], 'content': '告解解論', 'transliteration': 'cáo giải giải luận '}, {'bbox': [[799, 3293], [984, 3297], [976, 4188], [795, 4184]], 'content': '十七張', 'transliteration': 'thập thất trương '}, {'bbox': [[453, 2805], [641, 2805], [641, 3262], [453, 3262]], 'content': '目鑠', 'transliteration': 'mục thước '}]}, {'page_number': 1, 'content': [{'bbox': [[3271, 1471], [3425, 1471], [3425, 1944], [3271, 1944]], 'content': '男王', 'transliteration': 'nam vương '}, {'bbox': [[2929, 1164], [3194, 1168], [3179, 2320], [2917, 2317]], 'content': '告明補贖', 'transliteration': 'cáo minh bổ chuộc '}, {'bbox': [[3029, 3247], [3213, 3247], [3213, 4100], [3029, 4100]], 'content': '廾一張', 'transliteration': 'chấp nhất trương '}, {'bbox': [[2556, 1145], [2863, 1148], [2863, 2286], [2556, 2286]], 'content': '終傅神品', 'transliteration': 'chung phó thần phẩm '}, {'bbox': [[2667, 3235], [2887, 3235], [2887, 4119], [2667, 4119]], 'content': '卅三張', 'transliteration': 'táp tam trương '}, {'bbox': [[2252, 1141], [2514, 1141], [2514, 2324], [2252, 2324]], 'content': '十誠解義', 'transliteration': 'thập thành giải nghĩa '}, {'bbox': [[2337, 3262], [2529, 3262], [2529, 4134], [2337, 4134]], 'content': '廾七張', 'transliteration': 'nhập thất trương '}, {'bbox': [[1983, 434], [2033, 434], [2033, 522], [1983, 522]], 'content': '4', 'transliteration': '4 '}, {'bbox': [[1891, 1114], [2183, 1114], [2183, 2320], [1891, 2320]], 'content': '㑹聖條律', 'transliteration': 'hội thánh điều luật '}, {'bbox': [[2025, 3266], [2214, 3273], [2179, 4073], [1991, 4065]], 'content': '三十八', 'transliteration': 'tam thập bát '}, {'bbox': [[1556, 1141], [1829, 1141], [1829, 2297], [1556, 2297]], 'content': '天主經解', 'transliteration': 'thiên chủ kinh giải '}, {'bbox': [[1653, 3277], [1852, 3277], [1852, 4115], [1653, 4115]], 'content': '三十九', 'transliteration': 'tam thập cửu '}, {'bbox': [[1230, 1137], [1503, 1137], [1503, 2301], [1230, 2301]], 'content': '聖母經解', 'transliteration': 'thánh mẫu kinh giải '}, {'bbox': [[1349, 3250], [1526, 3254], [1514, 4096], [1341, 4092]], 'content': '四十四', 'transliteration': 'tứ thập tứ '}, {'bbox': [[864, 1152], [1160, 1152], [1160, 2301], [864, 2301]], 'content': '正役敎友', 'transliteration': 'chính việc giáo hữu '}, {'bbox': [[991, 3254], [1199, 3254], [1199, 4130], [991, 4130]], 'content': '四十八', 'transliteration': 'tứ thập bát '}, {'bbox': [[499, 1141], [791, 1133], [830, 2851], [538, 2858]], 'content': '經畧逴禮五十', 'transliteration': 'kinh lược rước lễ ngũ thập '}, {'bbox': [[530, 3216], [807, 3216], [807, 4403], [530, 4403]], 'content': '經娄連禮', 'transliteration': 'kinh lâu liền lễ '}, {'bbox': [[665, 4442], [807, 4442], [807, 4714], [665, 4714]], 'content': '五十', 'transliteration': 'ngũ thập '}, {'bbox': [[511, 4457], [645, 4457], [645, 4591], [511, 4591]], 'content': '二', 'transliteration': 'nhị '}, {'bbox': [[518, 4730], [807, 4730], [807, 4980], [518, 4980]], 'content': '終', 'transliteration': 'chung'}]}]

In [4]:
vn_content = [{'page_number': 0, 'content': 'lời giới thiệu\nthánh giáo yếu lý quốc ngữ được xuất bản lần đầu vào năm 1774 tại trung quốc đến năm 1933 thánh giáo yếu lý quốc ngữ được tái bản lần cuối\nthánh giáo yếu lý quốc ngữ được giới thiệu ở đây xuất bản năm 1837\ntác giả của thánh giáo yếu lý quốc ngữ là đức cha pigneau de belhaine 17411799 viết bằng chữ nôm\nnăm 1997 thánh giáo yếu lý quốc ngữ được nhóm dịch thuật hán nôm công giáo do cố linh mục vĩnh sơn nguyễn hưng phiên âm ghép trong quyển thánh giáo yếu lý và lưu hành nội bộ\nnay được phép của đức giám mục gioan đỗ văn ngân chủ tịch ủy ban giáo lý đức tin ban từ vựng hán nôm công giáo do đức ông phêrô nguyễn chí thiết chủ biên và thầy michel nguyễn hạnh hiệu đính chú thích và sắp xếp lại bố cục mộc bản chữ nôm ở trang trái theo cột từ phải sang trái và bản phiên âm việt ở trang phải sắp xếp theo dòng tương ứng với cột ở trang trái theo hướng từ trên xuống dưới\nnay kính\nđức ông phêrô nguyễn chí thiết\ntrưởng ban từ vựng hán nôm công giáo\n'}, {'page_number': 1, 'content': 'thánh giáo yếu lý\nmục lục\nthiên chúa nhất thể\ngiáng sinh cứu thế\ntử kỳ hữu định\nnhân sinh tội chủng\nthánh bí tích luận\nthánh thể giải luận\ncáo giải giải luận\n'}, {'page_number': 2, 'content': 'cáo minh bổ thục kiến nhất trang\nchung truyền thần phẩm kiến tam trang\nthập giới giải nghĩa kiến lục trang\nhội thánh điều luật kiến bát trang\nthiên chủ kinh giải thập nhất trang\nthánh mẫu kinh giải thập tứ trang\nchính dịch giáo hữu thập thất trang\nkinh trước rước lễ ngũ thập kinh sau rước lễ ngũ thập nhị chung\n'}]

In [5]:
sn2vn_content = list()
for page in sn_content:
    sn2vn_content.append({'page_number': page['page_number'], 'content': '\n'.join([line['transliteration'] for line in page['content']])})
    # align paragraphs
paragraph_alignments = alignment.paragraph_alignment(sn2vn_content,vn_content)

In [6]:
print(paragraph_alignments)


[{'source_page_number': 0, 'source_text': 'hình giáo yếu lý mục lục \nthiên chúa nhất thể \nkiến nhất trương \ngiáng sanh cứu thế \nkiến tam trương \n4 \ntử kỳ hữu định \nkiến lục trương \nnhân sinh tội giống \nkiến bát trương \nthánh bí tích luận \nthập nhất trương \nthánh thể giải luận \nthập tứ trương \ncáo giải giải luận \nthập thất trương \nmục thước ', 'target_page_number': 1, 'target_text': 'thánh giáo yếu lý\nmục lục\nthiên chúa nhất thể\ngiáng sinh cứu thế\ntử kỳ hữu định\nnhân sinh tội chủng\nthánh bí tích luận\nthánh thể giải luận\ncáo giải giải luận\n', 'similarity_score': 0.8878879547119141}, {'source_page_number': 1, 'source_text': 'nam vương \ncáo minh bổ chuộc \nchấp nhất trương \nchung phó thần phẩm \ntáp tam trương \nthập thành giải nghĩa \nnhập thất trương \n4 \nhội thánh điều luật \ntam thập bát \nthiên chủ kinh giải \ntam thập cửu \nthánh mẫu kinh giải \ntứ thập tứ \nchính việc giáo hữu \ntứ thập bát \nkinh lược rước lễ ngũ thập \nkinh lâu liền lễ \nngũ thập \nnhị 

In [35]:
import pandas as pd
from xlsxwriter.workbook import Workbook
result = list()
with Workbook(f"sample_output.xlsx") as workbook:
    worksheet   = workbook.add_worksheet(f"Result")
    font_format = workbook.add_format({'font_name': 'Nom Na Tong'})
    red         = workbook.add_format({'color': 'red', 'font_name': 'Nom Na Tong'})
    yellow      = workbook.add_format({'color': 'yellow', 'font_name': 'Nom Na Tong'})
    blue        = workbook.add_format({'color': 'blue', 'font_name': 'Nom Na Tong'})
    green       = workbook.add_format({'color': 'green', 'font_name': 'Nom Na Tong'})
    black       = workbook.add_format({'color': 'black', 'font_name': 'Nom Na Tong'})
    worksheet.write(0, 0, 'page_id', font_format)
    worksheet.write(0, 1, 'bbox', font_format)
    worksheet.write(0, 2, 'ocr', font_format)
    worksheet.write(0, 3, 'correction', font_format)
    worksheet.write(0, 4, 'nom', font_format)
    row = 1
    for idx, alignment in enumerate(paragraph_alignments[1:]):
        src_id = alignment['source_page_number']
        tgt_id = alignment['target_page_number']
        path_temp_file_sn2vn = os.path.join('data', "test.tr")
        path_temp_file_vn = os.path.join('data', "test.vn")
        with open(path_temp_file_sn2vn, "w", encoding="utf-8") as f:
            f.write(sn2vn_content[src_id]['content'])
        with open(path_temp_file_vn, "w", encoding="utf-8") as f:
            f.write(vn_content[tgt_id]['content'])
        os.chdir('./sentence_alignments')
        script = './align.sh' 
        print(f"Running: {script}")
        os.system(script)
        os.chdir('../')
        with open('data/output.txt','r',encoding='utf-8') as f:
            alignments=''.join(f.readlines()).split('\n')
        src_lines = [(line['bbox'],line['content']) for line in sn_content[src_id]['content']]
        tgt_lines = vn_content[tgt_id]['content'].split('\n')
        for i in range(len(alignments)-1):
            x = eval(alignments[i].split(':')[0])
            y = eval(alignments[i].split(':')[1])
            if len(x)>0 and len(y)>0:
                src_line = [(src_lines[j][0],src_lines[j][1]) for j in x]
                tgt_line = ' '.join([tgt_lines[j] for j in y]).split(' ')
                nom_line = ''.join([line[1] for line in src_line])
                corrected_list=crt.correct(nom_line,tgt_line)
                vie_list = tgt_line.copy()
                for chunk in src_line:
                    page_id = src_id
                    bbox = chunk[0]
                    nom = ''
                    corrected_nom = ''
                    vie = '' 
                    nom_list = list(chunk[1])
                    ocrs = []
                    corrs = []
                    qns = []
                    print(chunk[1],corrected_list)
                    while len(nom_list)>0:
                        if corrected_list[0].startswith('correct:'):
                            nom=nom+nom_list[0]
                            corrected_nom=corrected_nom+corrected_list[0].split(':')[1]
                            vie = vie + vie_list[0] + ' '
                            ocrs.extend((black, nom_list[0]))
                            corrs.extend((black, corrected_list[0].split(':')[1]))
                            qns.extend((black, vie_list[0] + ' '))
                            nom_list.pop(0)
                            vie_list.pop(0)
                        elif corrected_list[0].startswith('replace:'):
                            nom=nom+nom_list[0]
                            corrected_nom=corrected_nom+corrected_list[0].split(':')[1][-1]
                            vie = vie + vie_list[0] + ' '
                            ocrs.extend((green, nom_list[0]))
                            corrs.extend((green, corrected_list[0].split(':')[1][-1]))
                            qns.extend((green, vie_list[0] + ' '))
                            nom_list.pop(0)
                            vie_list.pop(0)
                        elif corrected_list[0].startswith('delete:'):
                            nom=nom+nom_list[0]
                            corrected_nom=corrected_nom+' '
                            vie = vie + ''
                            ocrs.extend((red, nom_list[0]))
                            nom_list.pop(0)
                        elif corrected_list[0].startswith('insert:'):
                            nom=nom+' '
                            corrected_nom=corrected_nom+corrected_list[0].split(':')[1]
                            vie = vie + vie_list[0] + ' '
                            corrs.extend((blue, corrected_list[0].split(':')[1]))
                            qns.extend((blue, vie_list[0] + ' '))
                            vie_list.pop(0)
                        corrected_list.pop(0)
                    result.append([page_id,bbox,nom,corrected_nom,vie])
                    worksheet.write(row, 0, page_id, font_format)
                    worksheet.write(row, 1, str(bbox), font_format)
                    worksheet.write_rich_string(row, 2, *ocrs)
                    worksheet.write_rich_string(row, 3, *corrs)
                    worksheet.write_rich_string(row, 4, *qns)
                    row =  row+1

Running: ./align.sh


../../LASER/tasks/embed/embed.sh: 64: [[: not found


2025-01-18 14:41:39,295 | INFO | embed | spm_model: /mnt/d/LASER//laser2.spm
2025-01-18 14:41:39,295 | INFO | embed | spm_cvocab: /mnt/d/LASER/laser2.cvocab
2025-01-18 14:41:39,295 | INFO | embed | loading encoder: /mnt/d/LASER//laser2.pt


/mnt/d/LASER/laser_encoders/models.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


2025-01-18 14:41:40,821 | INFO | embed | transfer encoder to GPU
2025-01-18 14:41:41,158 | INFO | preprocess | SPM processing overlaps.tr  
2025-01-18 14:41:41,352 | INFO | embed | encoding /tmp/tmpxfzfo1da/spm to ../data/overlaps.tr.emb
2025-01-18 14:41:41,613 | INFO | embed | encoded 141 sentences in 0s


../../LASER/tasks/embed/embed.sh: 64: [[: not found


2025-01-18 14:41:46,188 | INFO | embed | spm_model: /mnt/d/LASER//laser2.spm
2025-01-18 14:41:46,188 | INFO | embed | spm_cvocab: /mnt/d/LASER/laser2.cvocab
2025-01-18 14:41:46,188 | INFO | embed | loading encoder: /mnt/d/LASER//laser2.pt


/mnt/d/LASER/laser_encoders/models.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


2025-01-18 14:41:47,767 | INFO | embed | transfer encoder to GPU
2025-01-18 14:41:48,051 | INFO | preprocess | SPM processing overlaps.vn  
2025-01-18 14:41:48,264 | INFO | embed | encoding /tmp/tmprg70imdn/spm to ../data/overlaps.vn.emb
2025-01-18 14:41:48,484 | INFO | embed | encoded 37 sentences in 0s
告明補贖 ['correct:告', 'correct:明', 'correct:補', 'correct:贖', 'replace:廾->見', 'correct:一', 'correct:張']
[<xlsxwriter.format.Format object at 0x7fc138afa890>, 'cáo ', <xlsxwriter.format.Format object at 0x7fc138afa890>, 'minh ', <xlsxwriter.format.Format object at 0x7fc138afa890>, 'bổ ', <xlsxwriter.format.Format object at 0x7fc138afa890>, 'thục ']
廾一張 ['replace:廾->見', 'correct:一', 'correct:張']
[<xlsxwriter.format.Format object at 0x7fc138afbe50>, 'kiến ', <xlsxwriter.format.Format object at 0x7fc138afa890>, 'nhất ', <xlsxwriter.format.Format object at 0x7fc138afa890>, 'trang ']
終傅神品 ['correct:終', 'correct:傳', 'correct:神', 'correct:品', 'replace:卅->見', 'correct:三', 'correct:張']
[<xlsxwriter.

/home/davidle2810/anaconda3/envs/mm_align/lib/python3.10/site-packages/xlsxwriter/worksheet.py:1344: UserWarning: You must specify more than 2 format/fragments for rich strings. Ignoring input in write_rich_string().
  warn(


In [8]:
src_line='十誠解義廾七張'
tgt_line='thập giới giải nghĩa kiến lục trang'
print(crt.normalize_correction(crt.correct(src_line,tgt_line.split(' '))))

十誡解義廾七張


In [30]:
result

[[1,
  [[2929, 1164], [3194, 1168], [3179, 2320], [2917, 2317]],
  '告明補贖',
  '告明補贖',
  'cáo minh bổ thục '],
 [1,
  [[3029, 3247], [3213, 3247], [3213, 4100], [3029, 4100]],
  '廾一張',
  '見一張',
  'kiến nhất trang '],
 [1,
  [[2556, 1145], [2863, 1148], [2863, 2286], [2556, 2286]],
  '終傅神品',
  '終傳神品',
  'chung truyền thần phẩm '],
 [1,
  [[2667, 3235], [2887, 3235], [2887, 4119], [2667, 4119]],
  '卅三張',
  '見三張',
  'kiến tam trang '],
 [1,
  [[2252, 1141], [2514, 1141], [2514, 2324], [2252, 2324]],
  '十誠解義',
  '十誡解X',
  'thập giới giải nghĩa '],
 [1,
  [[2337, 3262], [2529, 3262], [2529, 4134], [2337, 4134]],
  '廾七張',
  '見绿張',
  'kiến lục trang '],
 [1,
  [[1891, 1114], [2183, 1114], [2183, 2320], [1891, 2320]],
  '㑹聖條律',
  '聵聖條律',
  'hội thánh điều luật '],
 [1,
  [[2025, 3266], [2214, 3273], [2179, 4073], [1991, 4065]],
  '三十八',
  '見 八',
  'kiến bát '],
 [1,
  [[1556, 1141], [1829, 1141], [1829, 2297], [1556, 2297]],
  '天主經解',
  '天主經解',
  'thiên chủ kinh giải '],
 [1,
  [[1653, 3277], [1